<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [13]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Version:  2.9.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [15]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [16]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [17]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [6]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [18]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [ ]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

In [19]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

'rm' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 2s 46ms/step - loss: 0.6669 - accuracy: 0.6022 - val_loss: 0.6822 - val_accuracy: 0.4585
Epoch 2/10
26/26 [==============================] - 1s 35ms/step - loss: 0.6114 - accuracy: 0.6861 - val_loss: 0.5657 - val_accuracy: 0.7317
Epoch 3/10
26/26 [==============================] - 1s 35ms/step - loss: 0.5306 - accuracy: 0.7871 - val_loss: 0.4713 - val_accuracy: 0.8537
Epoch 4/10
26/26 [==============================] - 1s 35ms/step - loss: 0.4869 - accuracy: 0.7908 - val_loss: 0.4671 - val_accuracy: 0.7610
Epoch 5/10
26/26 [==============================] - 1s 35ms/step - loss: 0.3965 - accuracy: 0.8698 - val_loss: 0.3131 - val_accuracy: 0.9366
Epoch 6/10
26/26 [==============================] - 1s 35ms/step - loss: 0.3005 - accuracy: 0.9173 - val_loss: 0.2412 - val_accuracy: 0.9415
Epoch 7/10
26/26 [==============================] - 1s 35ms/step - loss: 0.2397 - accuracy: 0.9319 - val_loss: 0.2884 - val_accuracy: 0.8780
Epoch 8/10
26

In [12]:
%tensorboard --logdir logs

Launching TensorBoard...

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [21]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 1: saving model to weights.01-0.68.h5
26/26 - 2s - loss: 0.6816 - accuracy: 0.5669 - val_loss: 0.6767 - val_accuracy: 0.5220 - 2s/epoch - 61ms/step
Epoch 2/5

Epoch 2: saving model to weights.02-0.62.h5
26/26 - 1s - loss: 0.6417 - accuracy: 0.6813 - val_loss: 0.6236 - val_accuracy: 0.7610 - 894ms/epoch - 34ms/step
Epoch 3/5

Epoch 3: saving model to weights.03-0.56.h5
26/26 - 1s - loss: 0.5931 - accuracy: 0.7056 - val_loss: 0.5625 - val_accuracy: 0.7805 - 911ms/epoch - 35ms/step
Epoch 4/5

Epoch 4: saving model to weights.04-0.49.h5
26/26 - 1s - loss: 0.5438 - accuracy: 0.7616 - val_loss: 0.4929 - val_accuracy: 0.8244 - 913ms/epoch - 35ms/step
Epoch 5/5

Epoch 5: saving model to weights.05-0.42.h5
26/26 - 1s - loss: 0.4628 - accuracy: 0.8224 - val_loss: 0.4169 - val_accuracy: 0.8390 - 894ms/epoch - 34ms/step


In [22]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 1: saving model to saved_model


INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


26/26 - 3s - loss: 0.6746 - accuracy: 0.5657 - val_loss: 0.7315 - val_accuracy: 0.4390 - 3s/epoch - 112ms/step


In [23]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 1: saving model to model.h5
26/26 - 1s - loss: 0.6880 - accuracy: 0.5572 - val_loss: 0.6626 - val_accuracy: 0.5707 - 1s/epoch - 56ms/step
Epoch 2/2

Epoch 2: saving model to model.h5
26/26 - 1s - loss: 0.6427 - accuracy: 0.6217 - val_loss: 0.6920 - val_accuracy: 0.5122 - 908ms/epoch - 35ms/step


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [24]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 1s - loss: 0.6698 - accuracy: 0.5852 - val_loss: 0.6700 - val_accuracy: 0.4780 - 1s/epoch - 55ms/step
Epoch 2/50
26/26 - 1s - loss: 0.6187 - accuracy: 0.6715 - val_loss: 0.6551 - val_accuracy: 0.5805 - 865ms/epoch - 33ms/step
Epoch 3/50
26/26 - 1s - loss: 0.5615 - accuracy: 0.7494 - val_loss: 0.6343 - val_accuracy: 0.5659 - 865ms/epoch - 33ms/step
Epoch 4/50
26/26 - 1s - loss: 0.5022 - accuracy: 0.7664 - val_loss: 0.4359 - val_accuracy: 0.8098 - 876ms/epoch - 34ms/step
Epoch 5/50
26/26 - 1s - loss: 0.4259 - accuracy: 0.8163 - val_loss: 0.3679 - val_accuracy: 0.8390 - 872ms/epoch - 34ms/step
Epoch 6/50
26/26 - 1s - loss: 0.3820 - accuracy: 0.8394 - val_loss: 0.3121 - val_accuracy: 0.8976 - 865ms/epoch - 33ms/step
Epoch 7/50
26/26 - 1s - loss: 0.3334 - accuracy: 0.8686 - val_loss: 0.6487 - val_accuracy: 0.6537 - 846ms/epoch - 33ms/step
Epoch 8/50
26/26 - 1s - loss: 0.2737 - accuracy: 0.9015 - val_loss: 0.2008 - val_accuracy: 0.9415 - 859ms/epoch - 33ms/step
Epoch 9/50


## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [25]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 1s 40ms/step - loss: 0.6671 - accuracy: 0.5973 - val_loss: 0.6375 - val_accuracy: 0.7220
Epoch 2/5
26/26 [==============================] - 1s 34ms/step - loss: 0.6097 - accuracy: 0.7007 - val_loss: 0.5836 - val_accuracy: 0.6683
Epoch 3/5
26/26 [==============================] - 1s 35ms/step - loss: 0.5605 - accuracy: 0.7251 - val_loss: 0.5183 - val_accuracy: 0.7756
Epoch 4/5
26/26 [==============================] - 1s 34ms/step - loss: 0.4982 - accuracy: 0.7737 - val_loss: 0.4531 - val_accuracy: 0.8195
Epoch 5/5
26/26 [==============================] - 1s 34ms/step - loss: 0.4271 - accuracy: 0.8370 - val_loss: 0.4053 - val_accuracy: 0.8780


In [26]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.597324,0.667123,0.721951,0.637480
1,1,0.700730,0.609724,0.668293,0.583598
2,2,0.725061,0.560527,0.775610,0.518265
3,3,0.773723,0.498158,0.819512,0.453138
4,4,0.836983,0.427106,0.878049,0.405260


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [27]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 1: LearningRateScheduler setting learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 2s 41ms/step - loss: 0.6685 - accuracy: 0.6192 - val_loss: 0.6835 - val_accuracy: 0.4585 - lr: 0.0050

Epoch 2: LearningRateScheduler setting learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 1s 35ms/step - loss: 0.6410 - accuracy: 0.6533 - val_loss: 0.6527 - val_accuracy: 0.6146 - lr: 0.0025

Epoch 3: LearningRateScheduler setting learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 1s 35ms/step - loss: 0.6246 - accuracy: 0.7141 - val_loss: 0.6365 - val_accuracy: 0.7366 - lr: 0.0012

Epoch 4: LearningRateScheduler setting learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 1s 35ms/step - loss: 0.6173 - accuracy: 0.7664 - val_loss: 0.6309 - val_accuracy: 0.7659 - lr: 6.2500e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 1s 

In [29]:
%tensorboard --logdir log_dir

Launching TensorBoard...

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [30]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 2s 41ms/step - loss: 0.6871 - accuracy: 0.5487 - val_loss: 0.6670 - val_accuracy: 0.6390 - lr: 0.0100
Epoch 2/50
26/26 [==============================] - 1s 35ms/step - loss: 0.6464 - accuracy: 0.6217 - val_loss: 0.6266 - val_accuracy: 0.7561 - lr: 0.0100
Epoch 3/50
26/26 [==============================] - 1s 35ms/step - loss: 0.6192 - accuracy: 0.6898 - val_loss: 0.6159 - val_accuracy: 0.6439 - lr: 0.0100
Epoch 4/50
26/26 [==============================] - 1s 35ms/step - loss: 0.5644 - accuracy: 0.7628 - val_loss: 0.5330 - val_accuracy: 0.7951 - lr: 0.0100
Epoch 5/50
26/26 [==============================] - 1s 35ms/step - loss: 0.5450 - accuracy: 0.7384 - val_loss: 0.4981 - val_accuracy: 0.7805 - lr: 0.0100
Epoch 6/50
25/26 [===========================>..] - ETA: 0s - loss: 0.5084 - accuracy: 0.7525
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 1s 35ms/step - loss

In [31]:
%tensorboard --logdir log_dir

Launching TensorBoard...